In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 使 GPU 0 和 1 可见

import sys
sys.path.append("..")
sys.path.append("../mango")
sys.path.append("../mango/design_baselines/diff")


import numpy as np
import torch
import pytorch_lightning as pl

import mango_jupyter_scripts
import design_bench
from helpers import seed_everything 

from SDE import DiffusionSOO



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
#!/usr/bin/env python3
"""
Main driver program for the mango as one of baseline methods for design-bench.

"""
import argparse
import logging
def build_args() -> argparse.Namespace:
    """
    Defines the experimental arguments for offline MBO baseline method
    evaluation.
    Input:
        None.
    Returns:
        A namespace containing the experimental argument values.
    """
    parser = argparse.ArgumentParser(description="Our method Experiments")

    parser.add_argument(
        "--task",
        type=str,
        default='AntMorphology-Exact-v0', 
        choices=[task.task_name for task in design_bench.registry.all()],
        help="The name of the design-bench task for the experiment."
    )
    parser.add_argument(
        "--logging-dir",
        type=str,
        default=None,
        help="Logging directory to save optimization results to."
    )
    parser.add_argument(
        "--budget", type=int, default=2048, help="Query budget. Default 2048."
    )
    parser.add_argument(
        "--hidden-size",
        type=int,
        default=2048,
        help="Hidden size of the surrogate objective model. Default 2048."
    )
    parser.add_argument(
        "--seed", type=int, default=2024, help="Random seed. Default 42."
    )
    parser.add_argument(
        "--device", type=str, default="auto", help="Device. Default `auto`."
    )
    args = parser.parse_args([])
    return args

args = build_args()


In [ ]:
from mango_jupyter_scripts.training import train_surrogate, mango_train
# Training a surrogate forward model (It is merely to provide a forward model for comparison purposes only).
forwardmodel = train_surrogate(task_name=args.task, hidden_size=args.hidden_size)

Loading model from ../checkpoints/surrogate/AntMorphology-Exact-v0-surrogate-100epoch.ckpt


In [6]:
task = design_bench.make( args.task)
normalize_x = True
normalize_y = True
x_min, x_max, y_min, y_max = np.min(task.dataset.x, axis=0), np.max(task.dataset.x, axis=0), np.min(task.dataset.y, axis=0), np.max(task.dataset.y, axis=0) 
try:
    x1_range = task.dataset.x1_range
    x2_range = task.dataset.x2_range
    x_lbound = np.array([x1_range[0], x2_range[0]])
    x_ubound = np.array([x1_range[1], x2_range[1]])
except:
    x_lbound = task.oracle.internal_dataset.x.min(axis=0)
    x_ubound = task.oracle.internal_dataset.x.max(axis=0)
    y_lbound = task.oracle.internal_dataset.y.min()
    y_ubound = task.oracle.internal_dataset.y.max()
    
print("##################Before Normalization##################")
print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

if normalize_y:
    task.map_normalize_y()
    y_min = task.normalize_y(y_min)[0]
    y_max = task.normalize_y(y_max)[0]
    y_lbound = task.normalize_y(y_lbound)[0]
    y_ubound = task.normalize_y(y_ubound)[0]
if normalize_x:
    task.map_normalize_x()
    x_min = task.normalize_x(x_min)[0]
    x_max = task.normalize_x(x_max)[0]
    x_lbound = task.normalize_x(x_lbound)[0]
    x_ubound = task.normalize_x(x_ubound)[0]

if normalize_x or normalize_y:
    print("##################After Normalization##################")
    print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
    print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
    print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

##################Before Normalization##################
x_min: [-3.4914367e-02 -8.1086578e-03 -1.1334186e-02 -3.6367794e+01
 -3.7077602e+01 -3.5632595e+01 -4.3471195e+01  1.9532245e+00
 -3.4045948e+01  2.6839813e+01  1.7082998e+01  1.5066694e+01
  1.7718686e-01  1.6345143e-01  3.4744915e-01 -2.6592629e-02
 -2.2959379e-02 -1.8237045e-02 -2.6770067e+01 -2.7498465e+01
  4.7541565e+01 -3.4308289e+01  1.2710465e+00 -2.5697126e+01
  2.3862228e+01  1.6057274e+01  1.5534728e+01  1.6454867e-01
  1.7966314e-01  3.0671242e-01 -2.0316277e-02 -2.6302628e-02
 -2.3783758e-02 -3.6136829e+01 -2.4792904e+01  1.5169789e+02
 -2.1998133e+01 -4.2479956e-01 -3.0228489e+01  2.7208126e+01
  1.5378370e+01  1.6140862e+01  1.6478844e-01  1.6975120e-01
  3.3562821e-01 -1.1959282e-02 -2.4191769e-02 -3.7877318e-02
 -2.7562862e+01 -3.1050905e+01 -1.1412610e+02 -4.1602684e+01
 -4.9592346e-01 -4.1937038e+01  2.6713327e+01  2.8534780e+00
  1.4474700e+01  1.7531781e-01  1.8378699e-01  3.2620859e-01] 
 x_max: [ 2.3215380

In [ ]:

seed_everything(args.seed)
torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore", message="The dataloader, .* does not have many workers.*")

try:
    # # 尝试访问 inverse_model
    inverse_model

except NameError:
    inverse_model = None  # 如果未定义，设置为 None

# inverse_model = None
num_epochs=  2000
description_here = 'nsmc'
args = build_args()
args.logging_dir = args.logging_dir or f"../logs/run_ours_{args.task}_seed{args.seed}_budget{args.budget}_hidden{args.hidden_size}_epoch{num_epochs}_betamax{0.05}"
print(args.logging_dir )

logging.basicConfig(
    level=logging.INFO,  # 设置最低日志级别
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # 设置日志格式
    handlers=[
        logging.FileHandler(f'{args.logging_dir}/running.log'),  # 将日志写入文件
        # logging.StreamHandler()  # 同时将日志输出到控制台
    ]
)
 

# logging_dir = args.logging_dir
# while os.path.exists(logging_dir):
#     logging_dir = f"{args.logging_dir}_v{version}"
#     version += 1
# os.makedirs(logging_dir, exist_ok=True)


# inverse_model = mango_train(
#     task=task,
#     inverse_model=inverse_model,
#     task_name=args.task,
#     ckpt_dir=args.logging_dir,
#     hidden_size=args.hidden_size,
#     device=args.device,
#     seed=args.seed,
#     condition_training=False,
#     num_epochs=num_epochs,
#     dropout=0.0,
#     augment=True,
#     lr=5e-5,
#     batch_size=64,
#     debais=True,
# )


Global seed set to 2024


../logs/run_ours_AntMorphology-Exact-v0_seed2024_budget2048_hidden2048_epoch2000_betamax0.05


In [ ]:
model_ckpt = f"/home/tzhouaq/ManGO_SOO/logs/run_ours_AntMorphology-Exact-v0_seed2024_budget2048_hidden2048_epoch2000_betamax0.05/ours-ant_morphology/ant_morphology-seed2024-hidden2048-score_matchingFalse.ckpt"
inverse_model = DiffusionSOO.load_from_checkpoint(
    model_ckpt,
    taskname=task.dataset_name,
    task=task,
    lr=1e-4,
    dropout_p=0.0,
    hidden_size=args.hidden_size,
    debais=True,
    augment=True,
    condition_training=False
)
 

uncondition_training T: {1.0}


In [ ]:
from mango_jupyter_scripts.eval_fks import mango_eval
# This cell is to conduct unconditional generation

augment = True
condition_training = False
 
if augment:

    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

num_samples = 248
 

res = mango_eval(
    task=task,
    task_name=args.task,
    forwardmodel=forwardmodel,
    inverse_model=inverse_model,
    device='auto',
    logging_dir=args.logging_dir,
    num_samples=num_samples,
    augment=augment,
    clip_dic=clip_dic,
    # gamma=0,
    condition_training=condition_training,
    num_steps=500,
    seed=args.seed,

)

res['x'].shape#, res['y'].shape, res['y_scores'].shape
 

{'simple_clip': True, 'clip_min': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64), 'clip_max': tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       dtype=torch.float64)}
the first 5 (no rank) design (normalized_x=True, normalized_y=True) is:
 tensor([[

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


(248, 60)

In [ ]:
# This cell is to show the k-shot results of unconditional generation
 

y_unob_max = task.oracle.internal_dataset.y.max()
y_unob_min = task.oracle.internal_dataset.y.min()

designs = res['x']
preds =   res['y_gen']
# preds = res['y']
scores = res['y_scores']


print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])


if task.is_normalized_y:
    best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
else:
    best_training_score = np.max(task.dataset.y)
print("Training dataset:\n The best score during training is: ",best_training_score )
oracle = np.max(scores)
print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
print(f' The corresponding design is: {designs[np.argmax(scores)]}')

k = 1
percentile = 100
one_shot = scores[np.argmax(preds)][0]
normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
 

k = 16
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

k = 128
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')
 

k = 256
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

Input structure:
 The number of the designs is:  60 ; the dimension of the designs is:  60
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [165.32640931]
Oracle results:
 On the given design: the Oracle is: 569.8955078125
 The corresponding design is: [-1.50608793e-02  1.11120390e-02 -2.61198729e-05  1.30583572e+00
  4.74939728e+00  6.72496414e+00 -1.44001007e+00  3.70519018e+00
 -1.50921803e+01  3.03115063e+01  4.62191353e+01  2.06458855e+01
  2.17444301e-01  1.98547214e-01  3.99040937e-01 -5.55733591e-03
  3.37109715e-03 -2.74186581e-03 -5.66118813e+00  3.57397652e+00
  9.31072540e+01 -9.11498642e+00  5.09014893e+00 -1.08057995e+01
  3.21095085e+01  6.09665642e+01  1.89588242e+01  2.01581329e-01
  2.03011483e-01  3.65873337e-01  3.92783433e-03  3.42843682e-03
  2.08310597e-03 -6.01292610e+00 -2.43004227e+00  1.88225494e+02
 -2.31624985e+00  2.03478909e+00 -9.50878525e+00  3.209890

In [ ]:
augment = True
if normalize_y:
    print('y_given before normalization:',  task.oracle.internal_dataset.y.max())
    y_given = torch.tensor(task.normalize_y(task.oracle.internal_dataset.y.max())[0])
    # y_given = torch.tensor(task.oracle.external_dataset.y.max()) 
else:
    y_given = torch.tensor(task.oracle.internal_dataset.y.max())
print('y_given:', y_given)
if y_max.item() < y_given:
    y_max = y_given 
if  y_min.item() > y_given:
    y_min = y_given

if augment:

    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

args.budget = 256

seed_list = list(range(args.seed, args.seed + 8))
seed_list = [ 39,40, 41,42,   44, 45, 46, 47, 49] 


reported_results_1shot = []
reported_results_16shot = []
reported_results_128shot = []
reported_results_256shot = []

args.budget = 256
num_steps = 500
inference_scaling_bool = False
duplicated_time = 64
guidance_scals2= 5
description = 'nonresampling'

# y_given = torch.tensor([2])
for seed in seed_list:
    print(f'***********************seed{seed}*************************')
    print(f"Running ours method for {args.task} with seed {seed}")
    
    res = mango_eval(
        task,
        task_name=args.task,
        forwardmodel=forwardmodel,
        inverse_model=inverse_model,
        logging_dir=args.logging_dir,
        hidden_size=args.hidden_size,
        num_samples=args.budget,
        device=args.device,
        seed=seed,
        clip_dic=clip_dic,
        augment=augment,
        gen_condition=y_given,
        guidance=True,
        num_steps=num_steps,
        inference_scaling_bool=inference_scaling_bool,
        duplicated_time=duplicated_time,
        guidance_scals2=guidance_scals2,
        description=description
    )
 
 
 
    y_unob_max = task.oracle.internal_dataset.y.max()
    y_unob_min = task.oracle.internal_dataset.y.min()

    designs = res['x']
    preds =   res['y_gen']
    # preds = res['y']
    scores = res['y_scores']
 
    print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
    print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

    
    # task = design_bench.make(args.task)
    if task.is_normalized_y:
        best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
    else:
        best_training_score = np.max(task.dataset.y)
    print("Training dataset:\n The best score during training is: ",best_training_score )
    oracle = np.max(scores)
    print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
    print(f' The corresponding design is: {designs[np.argmax(scores)]}')

    k = 1
    percentile = 100
    one_shot = scores[np.argmax(preds)][0]
    normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - normalize_one_shot))
    reported_results_1shot.append(normalize_one_shot)


    k = 16
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 128
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_128shot.append(normalize_few_shot)

    k = 256
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_256shot.append(normalize_few_shot)
    
print(f'***********************OVERALL RESULTS*************************')
print(f"reported_results_1shot: {np.mean(reported_results_1shot):.4f} ± {np.std(reported_results_1shot):.4f}")
print(f"reported_results_16shot: {np.mean(reported_results_16shot):.4f} ± {np.std(reported_results_16shot):.4f}")
print(f"reported_results_128shot: {np.mean(reported_results_128shot):.4f} ± {np.std(reported_results_128shot):.4f}")
print(f"reported_results_256shot: {np.mean(reported_results_256shot):.4f} ± {np.std(reported_results_256shot):.4f}")
 

 


y_given before normalization: 590.24445
y_given: tensor([1.0000], dtype=torch.float64)
{'simple_clip': True, 'clip_min': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64), 'clip_max': tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       dtype=torch.float